In [1]:
CONTEXT_SIZE = 2    # 依据的单词数
EMBEDDING_DIM = 10  # 词向量的维度
# 我们使用莎士比亚的诗
test_sentence = """When forty winters shall besiege thy brow,
And dig deep trenches in thy beauty's field,
Thy youth's proud livery so gazed on now,
Will be a totter'd weed of small worth held:
Then being asked, where all thy beauty lies,
Where all the treasure of thy lusty days;
To say, within thine own deep sunken eyes,
Were an all-eating shame, and thriftless praise.
How much more praise deserv'd thy beauty's use,
If thou couldst answer 'This fair child of mine
Shall sum my count, and make my old excuse,'
Proving his beauty by succession thine!
This were to be new made when thou art old,
And see thy blood warm when thou feel'st it cold.""".split()

In [2]:
# 构建训练集
trigram = [((test_sentence[i], test_sentence[i+1]), test_sentence[i+2]) 
            for i in range(len(test_sentence)-2)]

In [3]:
# 总的数据量
len(trigram)

113

In [4]:
# 取出第一个数据看看
trigram[0]

(('When', 'forty'), 'winters')

In [5]:
# 建立每个词与数字的编码（字典），据此构建词嵌入
vocb = set(test_sentence)  # 使用 set 将重复的元素去掉
word_to_idx = {word: i for i, word in enumerate(vocb)}
idx_to_word = {word_to_idx[word]: word for word in word_to_idx}

In [6]:
word_to_idx

{'where': 0,
 'forty': 1,
 'of': 2,
 'And': 3,
 'old,': 4,
 'the': 5,
 'Were': 6,
 'proud': 7,
 "'This": 8,
 'so': 9,
 'Then': 10,
 'cold.': 11,
 'gazed': 12,
 'treasure': 13,
 'sunken': 14,
 'be': 15,
 'on': 16,
 'How': 17,
 'This': 18,
 'field,': 19,
 'answer': 20,
 'dig': 21,
 'child': 22,
 'all': 23,
 'more': 24,
 'Proving': 25,
 'it': 26,
 'small': 27,
 'praise': 28,
 'all-eating': 29,
 'couldst': 30,
 'succession': 31,
 'in': 32,
 'see': 33,
 'If': 34,
 'warm': 35,
 'asked,': 36,
 'days;': 37,
 'thou': 38,
 'now,': 39,
 'and': 40,
 'his': 41,
 'lusty': 42,
 "deserv'd": 43,
 'old': 44,
 "beauty's": 45,
 'winters': 46,
 'Will': 47,
 'use,': 48,
 "excuse,'": 49,
 "youth's": 50,
 'much': 51,
 'worth': 52,
 'my': 53,
 'Thy': 54,
 'thine!': 55,
 'weed': 56,
 'Shall': 57,
 'besiege': 58,
 'beauty': 59,
 'mine': 60,
 'being': 61,
 'held:': 62,
 'new': 63,
 'when': 64,
 "feel'st": 65,
 'fair': 66,
 'a': 67,
 'own': 68,
 'sum': 69,
 'trenches': 70,
 'lies,': 71,
 'within': 72,
 'were': 73,

In [7]:
import torch
from torch import nn
from torch.autograd import Variable

In [8]:
class n_gram(nn.Module):
    def __init__(self, vocab_size, embedding_size, context_size):
        super(n_gram, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_size)
        self.fc = nn.Sequential(
                  nn.Linear(context_size * embedding_size, 128),
                  nn.ReLU(),
                  nn.Linear(128, vocab_size))
    
    def forward(self, x):
        embed = self.embedding(x)
        out = self.fc(embed.view(1, -1))
        return out

In [9]:
net = n_gram(len(vocb), EMBEDDING_DIM, CONTEXT_SIZE)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(net.parameters(), lr=0.1)

In [10]:
Epochs = 10
for epoch in range(Epochs):
    train_loss = 0.
    for words, label in trigram:
        words = Variable(torch.LongTensor([word_to_idx[word] for word in words]))
        label = Variable(torch.LongTensor([word_to_idx[label]]))
        y_ = net(words)
        loss = criterion(y_, label)
        train_loss += loss.detach().item()
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    train_loss /= len(trigram)
    print('epoch: {:2d}, train loss: {:.4f}'.format(epoch + 1, train_loss))

epoch:  1, train loss: 4.8617
epoch:  2, train loss: 3.5266
epoch:  3, train loss: 2.1766
epoch:  4, train loss: 0.9823
epoch:  5, train loss: 0.4030
epoch:  6, train loss: 0.1698
epoch:  7, train loss: 0.1136
epoch:  8, train loss: 0.0984
epoch:  9, train loss: 0.0874
epoch: 10, train loss: 0.0806


In [11]:
# 测试一下
net.eval()
words, label = trigram[80]
print('input words are: {}, the real word is: {}'.format(words, label))
index = Variable(torch.LongTensor([word_to_idx[word] for word in words]))
y_pred = net(index)
_, out = y_pred.max(1)
out = out.item()
real_word = idx_to_word[out]
print('predicted word is: {}'.format(real_word))

input words are: ('Shall', 'sum'), the real word is: my
predicted word is: my
